In [1]:

import os 
import numpy as np 
import h5py 
import json
import shutil
import time 
import torch 
import glob 

from tqdm import tqdm
from datetime import datetime
from utils import *
from constrained_inversion import *

# torch computation device to calculate jacobian via autograd
device = torch.device('cuda:0')


/home/nikolai/dev/constrained_inversion_mrm/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# get parameters of pulses to design
exp1_parfile = 'params_selective.json'
with open(exp1_parfile, 'r') as fid: par = json.load(fid)

# create output directory
exp1_processed_dir = os.path.join('output', 'sel_'+datetime.now().strftime("%Y%m%d_%H%M%S"))
os.system('mkdir -p %s'%(exp1_processed_dir))
shutil.copy(exp1_parfile, exp1_processed_dir)

# number of pulses to optimize 
num_pulses = len(par)

for p in tqdm(range(num_pulses)):

    # number of points in RF waveform
    num_samples = int(par[p]['dur'] / par[p]['raster_time'])

    # b1 scaling factors
    b1_scales = np.linspace(min(par[p]['b1+_range']), max(par[p]['b1+_range']), par[p]['num_b1+'], dtype=np.float64)

    # calculate the basis functions 
    basis, s = get_hs_basis(num_samples, par[p]['basis']['num_hs'], par[p]['basis']['dur_hs'], par[p]['basis']['mu_hs'], par[p]['basis']['beta_hs'], ranseed=par[p]['basis']['ranseed'])

    # get the target magnetization profile 
    freq_arr = np.linspace(-par[p]['bandwidth'], par[p]['bandwidth'], par[p]['num_offres'], dtype=np.float64)
    rf_hs = sech_waveform(par[p]['hs_for_reference']['dur'], num_samples, par[p]['bandwidth'], par[p]['hs_for_reference']['beta'])
    overdrive = 1 / min(par[p]['b1+_range'])
    rf_hs = scale_adiabatic_inv_to_overdrive_factor(rf_hs, overdrive, par[p]['raster_time'], device=device)
    mz_hs = simulate_mz(rf_hs, freq_arr, b1_scales, par[p]['raster_time'], device=device, return_numpy=True)

    # get the target magnetization profile 
    bw = par[p]['bandwidth']
    tw = par[p]['transition_width']
    freq_mask = np.zeros_like(freq_arr)
    mz_target = np.zeros((freq_arr.size, b1_scales.size), dtype=np.float64)
    mz_target[freq_arr < -0.5*bw - 0.5*tw,:] = 1.0
    freq_mask[freq_arr < -0.5*bw - 0.5*tw] = 1.0
    mz_target[freq_arr >  0.5*bw + 0.5*tw,:] = 1.0
    freq_mask[freq_arr > 0.5*bw + 0.5*tw] = 1.0
    freq_mask[(freq_arr > -0.5*bw + 0.5*tw) & (freq_arr <  0.5*bw - 0.5*tw)] = 1.0 if par[p]['passband_weight'] is None else par[p]['passband_weight']
    mz_target[(freq_arr > -0.5*bw + 0.5*tw) & (freq_arr <  0.5*bw - 0.5*tw),:] = -1.0

    # calculate the maximum energy of the pulse to be optimized
    E_hs = np.sum(np.abs(rf_hs * rf_hs.conj())*par[p]['raster_time'])
    max_energy = E_hs * par[p]['max_energy_fraction_of_hs_reference']
    
    # inittial guess will be HS pulse
    rf_init = rf_hs

    # design the pulse!
    start_time = time.time()
    rf_opt, mz_opt, x_opt = optimize_inversion_pulse_autograd(
        rf_init=rf_init,
        freq_arr=freq_arr,
        freq_mask=freq_mask,
        subspace=basis[:,:par[p]['basis']['rank']],
        mz_target=mz_target,
        duration=par[p]['dur'],
        max_rf_energy_hz_sqrd_sec=max_energy,
        max_nominal_b1_hz=par[p]['b1_tx_peak']*1e-6*par[p]['gamma_bar'],
        b1_scale_arr=b1_scales,
        sqp_max_iters=par[p]['sqp_max_iters'],
        device=device
    )
    comp_time = time.time() - start_time 
    
    # save the pulse
    pulse_file = os.path.join(exp1_processed_dir, 'pulse_%03d.h5'%(p))
    with h5py.File(pulse_file,'w') as F:
        F.create_dataset('basis', data=basis)
        F.create_dataset('s', data=s)
        F.create_dataset('rf_hs', data=rf_hs)
        F.create_dataset('rf_init', data=rf_init) 
        F.create_dataset('rf_opt', data=rf_opt)
        F.create_dataset('mz_opt', data=mz_opt)
        F.create_dataset('mz_hs', data=mz_hs)
        F.create_dataset('b1_scales', data=b1_scales)
        F.create_dataset('mz_target', data=mz_target)
        F.create_dataset('freq_arr', data=freq_arr)
        F.create_dataset('comp_time', data=comp_time)

 33%|███▎      | 1/3 [26:58<53:56, 1618.49s/it]

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.4285211774413439
       x: [-4.830e+03  5.754e+03 ... -1.422e+01 -5.306e+00]
     nit: 750
     jac: [ 1.808e-04 -2.019e-04 ... -9.865e-06 -1.169e-05]
    nfev: 839
    njev: 750


 67%|██████▋   | 2/3 [1:01:21<31:19, 1879.86s/it]

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.15763096923161868
       x: [ 1.300e+02 -5.806e+03 ... -3.661e+01 -6.826e+01]
     nit: 965
     jac: [-7.962e-06  6.649e-05 ...  3.299e-06  4.307e-06]
    nfev: 1125
    njev: 965


100%|██████████| 3/3 [1:30:05<00:00, 1801.76s/it]

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.08167739379257336
       x: [ 1.475e+03 -6.708e+03 ... -5.059e+01 -6.568e+01]
     nit: 809
     jac: [-9.984e-06  3.127e-05 ... -8.085e-06  7.350e-06]
    nfev: 868
    njev: 809


In [2]:

# get parameters of pulses to design
exp2_parfile = 'params_nonsel.json'
with open(exp2_parfile, 'r') as fid: par = json.load(fid)

# create output directory
exp2_processed_dir = os.path.join('output', 'nonsel_'+datetime.now().strftime("%Y%m%d_%H%M%S"))
os.system('mkdir -p %s'%(exp2_processed_dir))
shutil.copy(exp2_parfile, exp2_processed_dir)

# number of pulses to optimize 
num_pulses = len(par)

for p in tqdm(range(num_pulses)):

    # number of points in RF waveform
    num_samples = int(par[p]['dur'] / par[p]['raster_time'])

    # b1 scaling factors
    b1_scales = np.linspace(min(par[p]['b1+_range']), max(par[p]['b1+_range']), par[p]['num_b1+'], dtype=np.float64)

    # calculate the basis functions 
    basis, s = get_hs_basis(num_samples, par[p]['basis']['num_hs'], par[p]['basis']['dur_hs'], par[p]['basis']['mu_hs'], par[p]['basis']['beta_hs'], ranseed=par[p]['basis']['ranseed'])

    # array of off-resonance locations to optimize over 
    freq_arr = np.linspace(-par[p]['bandwidth']/2, par[p]['bandwidth']/2, par[p]['num_offres'], dtype=np.float64)

    # get the initial guess and maximum energy
    rf_init = sech_waveform(par[p]['hs_for_reference']['dur'], num_samples, par[p]['bandwidth']*1.5, par[p]['hs_for_reference']['beta'])
    overdrive = 1 / min(par[p]['b1+_range'])
    rf_init = scale_adiabatic_inv_to_overdrive_factor(rf_init, overdrive, par[p]['raster_time'], device=device)
    max_energy = np.sum(np.abs(rf_init * rf_init.conj())*par[p]['raster_time']) # allow it to be that of a hard pulse with maxed-out peak B1

    # get the target magnetization profile 
    freq_mask = np.ones_like(freq_arr)
    mz_target = -1.0 * np.ones((freq_arr.size, b1_scales.size), dtype=np.float64)

    # design the pulse!
    start_time = time.time()
    rf_opt, mz_opt, x_opt = optimize_inversion_pulse_autograd(
        rf_init=rf_init,
        freq_arr=freq_arr,
        freq_mask=freq_mask,
        subspace=basis[:,:par[p]['basis']['rank']],
        mz_target=mz_target,
        duration=par[p]['dur'],
        max_rf_energy_hz_sqrd_sec=max_energy,
        max_nominal_b1_hz=par[p]['b1_tx_peak']*1e-6*par[p]['gamma_bar'],
        b1_scale_arr=b1_scales,
        sqp_max_iters=par[p]['sqp_max_iters'],
        device=device
    )
    comp_time = time.time() - start_time 

    # find the minimum-duration HS pulse that achieves adiabaticity at lowest B1+ with prescribed peak tx B1
    hs_dur = 0.007
    dur_inc = 2*par[p]['raster_time']
    rf_hs = par[p]['gamma_bar'] * 1e-6 * par[p]['b1_tx_peak'] * sech_waveform(hs_dur, int(hs_dur/par[p]['raster_time']), par[p]['bandwidth']*4, par[p]['hs_for_reference']['beta'])
    while np.mean(simulate_mz(rf_hs, freq_arr, b1_scales, par[p]['raster_time'], device=device, return_numpy=True)) > np.mean(mz_opt):
        hs_dur += dur_inc 
        rf_hs = par[p]['gamma_bar'] * 1e-6 * par[p]['b1_tx_peak'] * sech_waveform(hs_dur, int(hs_dur/par[p]['raster_time']), par[p]['bandwidth']*4, par[p]['hs_for_reference']['beta'])
    mz_hs = simulate_mz(rf_hs, freq_arr, b1_scales, par[p]['raster_time'], device=device, return_numpy=True)
    print(hs_dur)
    
    # save the pulse
    pulse_file = os.path.join(exp2_processed_dir, 'pulse_%03d.h5'%(p))
    with h5py.File(pulse_file,'w') as F:
        F.create_dataset('basis', data=basis)
        F.create_dataset('s', data=s)
        F.create_dataset('rf_hs', data=rf_hs)
        F.create_dataset('rf_init', data=rf_init) 
        F.create_dataset('rf_opt', data=rf_opt)
        F.create_dataset('mz_opt', data=mz_opt)
        F.create_dataset('mz_hs', data=mz_hs)
        F.create_dataset('b1_scales', data=b1_scales)
        F.create_dataset('mz_target', data=mz_target)
        F.create_dataset('freq_arr', data=freq_arr)
        F.create_dataset('comp_time', data=comp_time)
        F.create_dataset('hs_dur', data=hs_dur)

  0%|          | 0/1 [00:00<?, ?it/s]

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.028425846751897205
       x: [ 4.331e+03  4.120e+03 ... -3.981e+01  8.566e+00]
     nit: 754
     jac: [-1.377e-05 -1.135e-05 ...  1.716e-05  9.998e-07]
    nfev: 764
    njev: 754


100%|██████████| 1/1 [04:07<00:00, 247.90s/it]

0.0073040000000000075
